## A tour of MLJ

### Models, machines, basic training and testing

Let's load data and define train and test rows:

In [1]:
using Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
using MLJ
using DataFrames, Statistics

Xraw = rand(300,3)
y = exp(Xraw[:,1] - Xraw[:,2] - 2Xraw[:,3] + 0.1*rand(300))
X = DataFrame(Xraw)

train, test = partition(eachindex(y), 0.70); # 70:30 split

┌ Info: Recompiling stale cache file /home/ayush99/.julia/compiled/v1.1/MLJ/rAU56.ji for MLJ [add582a8-e3ab-11e8-2d5e-e98b27df1bc7]
└ @ Base loading.jl:1184


A *model* is a container for hyperparameters:

In [3]:
knn_model=KNNRegressor(K=10)

KNNRegressor(K = 10,
             metric = MLJ.KNN.euclidean,
             kernel = MLJ.KNN.reciprocal,) @ 1…33

Wrapping the model in data creates a *machine* which will store training outcomes.

In [4]:
knn = machine(knn_model, X, y)

Machine{KNNRegressor} @ 7…12


Training on the training rows and evaluating on the test rows:

In [5]:
fit!(knn, rows=train)
yhat = predict(knn, X[test,:])
rms(yhat, y[test])

┌ Info: Training Machine{KNNRegressor} @ 7…12.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/machines.jl:102


0.056181310146433745

Or, we could have skipped the train/test definitions and evaluated one line:

In [6]:
evaluate!(knn, resampling=Holdout(fraction_train=0.7), measure=rms)

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/resampling.jl:91


0.056181310146433745

Our machine/model constructions and associateed fit/predict syntax anticipates a powerful extension for building networks of learners described later. Changing a hyperparameter and re-evaluating:

In [7]:
knn_model.K = 20
evaluate!(knn, resampling=Holdout(fraction_train=0.7))  # `default_measure(knn) == rms` so `measure` kwarg can be dropped

┌ Info: Evaluating using a holdout set. 
│ fraction_train=0.7 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/resampling.jl:91


0.07634238032999312

### Homogeneous ensembles

Here's a bagged ensemble model for 20 K-nearest neighbour regressors:

In [8]:
ensemble_model = EnsembleModel(atom=knn_model, n=20)

MLJ.DeterministicEnsembleModel(atom = KNNRegressor{} @ 1…33,
                               weights = Float64[],
                               bagging_fraction = 0.8,
                               rng = MersenneTwister(UInt32[0x9aabcac1, 0x23fe85ec, 0xdb9c6bcc, 0xba9f2529]),
                               n = 20,
                               parallel = true,
                               out_of_bag_measure = Any[],) @ 7…29

In [9]:
@more

MLJ.DeterministicEnsembleModel(atom = KNNRegressor(K = 20,
                                                   metric = MLJ.KNN.euclidean,
                                                   kernel = MLJ.KNN.reciprocal,),
                               weights = Float64[],
                               bagging_fraction = 0.8,
                               rng = MersenneTwister(UInt32[0x9aabcac1, 0x23fe85ec, 0xdb9c6bcc, 0xba9f2529]),
                               n = 20,
                               parallel = true,
                               out_of_bag_measure = Any[],) @ 7…29

It can be trained and tested the same as any other model:

In [10]:
ensemble = machine(ensemble_model, X, y)
estimates = evaluate!(ensemble, resampling=CV())

┌ Info: Evaluating using cross-validation. 
│ nfolds=6. 
│ shuffle=false 
│ measure=MLJ.rms 
│ operation=StatsBase.predict 
│ Resampling from all rows. 
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/resampling.jl:135
Cross-validating: 100%[=========================] Time: 0:00:03


6-element Array{Float64,1}:
 0.12482743153258992
 0.07508734859631627
 0.15502281894894274
 0.10822915364474125
 0.07549563178337156
 0.0791335239392    

In [11]:
mean(estimates)

0.10296598474086031

### Systematic tuning

Let's simultaneously tune the ensemble's `bagging_fraction` and the K-nearest neighbour hyperparameter `K`. Since one of our models is a field of the other, we have nested hyperparameters:

In [12]:
params(ensemble_model) # extract model hyperparameters as nested named tuple

(atom = (K = 20,
         metric = MLJ.KNN.euclidean,
         kernel = MLJ.KNN.reciprocal,),
 weights = Float64[],
 bagging_fraction = 0.8,
 rng = MersenneTwister(UInt32[0x9aabcac1, 0x23fe85ec, 0xdb9c6bcc, 0xba9f2529]),
 n = 20,
 parallel = true,
 out_of_bag_measure = Any[],)

To define a tuning grid, we construct ranges for the two parameters and collate these ranges following the same pattern above (omitting parameters that don't change):

In [13]:
B_range = range(ensemble_model, :bagging_fraction, lower= 0.5, upper=1.0, scale = :linear)
K_range = range(knn_model, :K, lower=1, upper=100, scale=:log10)
nested_ranges = (atom = (K = K_range,), 
                 bagging_fraction = B_range)

(atom = (K = NumericRange{K} @ 1…84,),
 bagging_fraction = NumericRange{bagging_fraction} @ 1…67,)

Now we choose a tuning strategy:

In [14]:
tuning = Grid(resolution=12)

Grid(resolution = 12,
     parallel = true,) @ 1…44

And a resampling strategy:

In [15]:
resampling = Holdout(fraction_train=0.8)

Holdout(fraction_train = 0.8,
        shuffle = false,) @ 6…63

And define a new model which wraps the these strategies around our ensemble model:

In [16]:
tuned_ensemble_model = TunedModel(model=ensemble_model, 
    tuning=tuning, resampling=resampling, nested_ranges=nested_ranges)

MLJ.DeterministicTunedModel(model = DeterministicEnsembleModel{KNNRegressor} @ 7…29,
                            tuning = Grid{} @ 1…44,
                            resampling = Holdout{} @ 6…63,
                            measure = nothing,
                            operation = StatsBase.predict,
                            nested_ranges = (atom = (K = NumericRange{K} @ 1…84,), bagging_fraction = NumericRange{bagging_fraction} @ 1…67),
                            minimize = true,
                            full_report = true,
                            train_best = true,) @ 1…72

Fitting the corresponding machine tunes the underlying model (in this case an ensemble) and retrains on all supplied data:

In [17]:
tuned_ensemble = machine(tuned_ensemble_model, X[train,:], y[train])
fit!(tuned_ensemble);

┌ Warning: No measure specified. Using measure=MLJ.rms. 
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/machines.jl:91
┌ Info: Training Machine{DeterministicTunedModel} @ 8…04.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/machines.jl:102
Iterating over a 132-point grid: 100%[=========================] Time: 0:00:04
┌ Info: Training best model on all supplied data.
└ @ MLJ /home/ayush99/.julia/packages/MLJ/LrTgW/src/tuning.jl:200


For each fitted machine, one may inspect a user-friendly version of the fitted parameters (as opposed to the hyperparameters stored in its model). In the current case this is the best ensemble model (trained on all available data):

In [18]:
fitted_params(tuned_ensemble)

(best_model = DeterministicEnsembleModel{KNNRegressor} @ 8…96,)

In [19]:
ans.best_model |> params

(atom = (K = 2,
         metric = MLJ.KNN.euclidean,
         kernel = MLJ.KNN.reciprocal,),
 weights = Float64[],
 bagging_fraction = 0.5454545454545454,
 rng = MersenneTwister(UInt32[0x9aabcac1, 0x23fe85ec, 0xdb9c6bcc, 0xba9f2529]),
 n = 20,
 parallel = true,
 out_of_bag_measure = Any[],)

The `report` method gives more detail on the tuning process:

In [20]:
report(tuned_ensemble) 

(parameter_names = ["atom.K" "bagging_fraction"],
 parameter_scales = Symbol[:log10 :linear],
 parameter_values = Any[1 0.5; 2 0.5; … ; 66 1.0; 100 1.0],
 measurements = [0.10563, 0.0848966, 0.0984288, 0.0997608, 0.118416, 0.124203, 0.151223, 0.187434, 0.218704, 0.266869  …  0.124131, 0.0831967, 0.073591, 0.0800684, 0.100143, 0.119439, 0.12947, 0.155378, 0.188244, 0.231607],
 best_measurement = 0.06895626545206299,)

Or for a plot of tuning results:

In [ ]:
using Plots
pyplot()
plot(tuned_ensemble)

Evaluating the model:

In [ ]:
yhat = predict(tuned_ensemble, X[test,:])
rms(yhat, y[test])

Or, using all the data, get cross-validation estimates, with cv-tuning on each fold complement (nested resampling):

In [ ]:
tuned_ensemble = machine(tuned_ensemble_model, X, y)
estimates = evaluate!(tuned_ensemble, resampling=CV(nfolds=4), verbosity=2)

In [ ]:
mean(estimates)

### Learning networks

MLJ has a flexible interface for building networks from multiple machine learning elements, whose complexity extend beyond linear "pipelines", and with a minimal of added abstraction.

In MLJ, a *learning network* is a graph whose nodes apply an operation, such as `predict` or `transform`, using a fixed machine (requiring training) - or which, alternatively, applies a regular (untrained) mathematical operation to its input(s). In practice, a learning network works with *fixed* sources for its training/evaluation data, but can be built and tested in stages. By contrast, an *exported learning network* is a learning network exported as a stand-alone, re-usable `Model` object, to which all the MLJ `Model`  meta-algorthims can be applied (ensembling, systematic tuning, etc). 

As we shall see, exporting a learning network as a reusable model, is very easy. 

### Building a simple learning network

![](wrapped_ridge.png)

The diagram above depicts a learning network which standardises the input data, `X`, learns an optimal Box-Cox transformation for the target, `y`, predicts new targets using ridge regression, and then inverse-transforms those predictions (for later comparison with the original test data). The machines are labelled yellow. 

To implement the network, we begin by loading all data needed for training and evaluation into *source nodes*:

In [ ]:
Xs = source(X)
ys = source(y)

We label nodes according to their outputs in the diagram. Notice that the nodes `z` and `yhat` use the same machine `box` for different operations. 

To construct the `W` node we first need to define the machine `stand` that it will use to transform inputs. 

In [ ]:
stand_model = Standardizer()
stand = machine(stand_model, Xs)

Because `Xs` is a node, instead of concrete data, we can call `transform` on the machine without first training it, and the result is the new node `W`, instead of concrete transformed data:

In [ ]:
W = transform(stand, Xs)

To get actual transformed data we *call* the node appropriately, which will require we first train the node. Training a node, rather than a machine, triggers training of *all* necessary machines in the network.

In [ ]:
fit!(W, rows=train)
W()          # transform all data
W(rows=test) # transform only test data
W(X[3:4,:])  # transform any data, new or old

If you like, you can think of `W` (and the other nodes we will define) as "dynamic data": `W` is *data*, in the sense that  it an be called ("indexed") on rows, but *dynamic*, in the sense the result depends on the outcome of training events. 

The other nodes of our network are defined similarly:

In [ ]:
box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
box = machine(box_model, ys)
z = transform(box, ys)

@load RidgeRegressor

ridge_model = RidgeRegressor(lambda=0.1)
ridge =machine(ridge_model, W, z)
zhat = predict(ridge, W)

yhat = inverse_transform(box, zhat)

We are ready to train and evaluate the completed network. Notice that the standardizer, `stand`, is *not* retrained, as MLJ remembers that it was trained earlier:

In [ ]:
fit!(yhat, rows=train)
rms(y[test], yhat(rows=test)) # evaluate

In [ ]:
yhat(X[3:4,:])  # predict on new or old data

We can change hyperparameters and retrain:

In [ ]:
ridge_model.lambda = 0.01
fit!(yhat, rows=train) 
rms(y[test], yhat(rows=test))

> **Notable feature.** The machine, `ridge::NodalMachine{RidgeRegressor}`, is retrained, because its underlying model has been mutated. However, since the outcome of this training has no effect on the training inputs of the machines `stand` and `box`, these transformers are left untouched. (During construction, each node and machine in a learning network determines and records all machines on which it depends.) This behaviour, which extends to exported learning networks, means we can tune our wrapped regressor without re-computing transformations each time the hyperparameter is changed. 

### Exporting a learning network as a composite model

To export a learning network:
- Define a new `mutable struct` model type.
- Wrap the learning network code in a model `fit` method.

All learning networks that make determinisic (or, probabilistic) predictions export as models of subtype `DeterministicNetwork` (respectively, `ProbabilisticNetwork`):


In [ ]:
mutable struct WrappedRidge <: DeterministicNetwork
    ridge_model
end

Now satisfied that our wrapped Ridge Regression learning network works, we simply cut and paste its defining code into a `fit` method: 

In [ ]:
function MLJ.fit(model::WrappedRidge, X, y)
    Xs = source(X)
    ys = source(y)

    stand_model = Standardizer()
    stand = machine(stand_model, Xs)
    W = transform(stand, Xs)

    box_model = UnivariateBoxCoxTransformer()  # for making data look normally-distributed
    box = machine(box_model, ys)
    z = transform(box, ys)

    ridge_model = model.ridge_model ###
    ridge =machine(ridge_model, W, z)
    zhat = predict(ridge, W)

    yhat = inverse_transform(box, zhat)
    fit!(yhat, verbosity=0)
    
    return yhat
end

The line marked `###`, where the new exported model's hyperparameter `ridge_model` is spliced into the network, is the only modification.

This completes the export process.

> **What's going on here?** MLJ's machine interface is built atop a more primitive *[model](adding_models_for_general_use.md)* interface, implemented for each algorithm. Each supervised model type (eg, `RidgeRegressor`) requires model `fit` and `predict` methods, which are called by the corresponding machine `fit!` and `predict` methods. We don't need to define a  model `predict` method here because MLJ provides a fallback which simply calls the node returned by `fit` on the data supplied: `MLJ.predict(model::Supervised{Node}, Xnew) = yhat(Xnew)`.

Let's now let's wrap our composite model as a tuned model and evaluate on the Boston dataset:

In [ ]:
task = load_boston()
X, y = task()
train, test = partition(eachindex(y), 0.7)
wrapped_model = WrappedRidge(ridge_model)

In [ ]:
params(wrapped_model)

In [ ]:
nested_ranges = (ridge_model = (lambda = range(ridge_model, :lambda, lower=0.1, upper=100.0, scale=:log10),),)

In [ ]:
tuned_wrapped_model = TunedModel(model=wrapped_model, tuning=Grid(resolution=20),
resampling=CV(), measure=rms, nested_ranges=nested_ranges);

In [ ]:
tuned_wrapped = machine(tuned_wrapped_model, X, y)
evaluate!(tuned_wrapped, resampling=Holdout(fraction_train=0.7), measure=rms, verbosity=0) |> mean  # nested resampling estimate